In [22]:
from numpy import array
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
from keras.utils import to_categorical
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.optimizers import SGD

import pandas as pd
from sodapy import Socrata



In [2]:
#Uncomment the below to open data source, download sensor data, and store it as a csv locally.

##Function to get Sensor count history data
# def sensor_count():
#     client = Socrata('data.melbourne.vic.gov.au', 'nlPM0PQJSjzCsbVqntjPvjB1f', None)
#     sensor_data_id = "b2ak-trbp"
#     results = client.get(sensor_data_id, limit=5000000)
#     df = pd.DataFrame.from_records(results)
#     df = df[['date_time', 'year', 'month', 'mdate', 'day', 'time', 'sensor_id', 'sensor_name', 'hourly_counts']]
#     return df

# sensor_history = sensor_count()

# sensor_history.to_csv('sensor_history.csv', index=False)

In [50]:
sensor_history = pd.read_csv('sensor_history.csv')

In [57]:
single_sensor = sensor_history[sensor_history.sensor_id == 4].copy()

In [58]:
#Make sensor counts range from 0 to 1.
maximum = single_sensor.hourly_counts.max()
single_sensor['hourly_counts'] = single_sensor['hourly_counts']/maximum

In [59]:
sensor_seq = []
for i in range(0, len(single_sensor)):
    sensor_seq.append(single_sensor.hourly_counts.iloc[i])

In [60]:
# split a univariate sequence
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# choose a number of time steps
n_steps = 14
# split into samples
X, y = split_sequence(sensor_seq, n_steps)


In [61]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))


In [62]:
epochs = 50
learning_rate = 0.001
decay_rate = learning_rate / epochs
momentum = 0.8

# define model
model = Sequential()
# model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
# model.add(Dropout(0.2))
# model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, n_features)))
model.add(LSTM(100, return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(Dense(13, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

opt = keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(optimizer=opt, loss='mse')


In [63]:
# fit model
model.fit(X, y, epochs=50, verbose=1)


Epoch 1/50
3368/3368 [==============================] - 50s 15ms/step - loss: 0.0087
Epoch 2/50
3368/3368 [==============================] - 51s 15ms/step - loss: 0.0011
Epoch 3/50
3368/3368 [==============================] - 51s 15ms/step - loss: 9.9722e-04
Epoch 4/50
3368/3368 [==============================] - 50s 15ms/step - loss: 9.0063e-04
Epoch 5/50
3368/3368 [==============================] - 49s 15ms/step - loss: 8.4081e-04
Epoch 6/50
3368/3368 [==============================] - 48s 14ms/step - loss: 8.0537e-04
Epoch 7/50
3368/3368 [==============================] - 48s 14ms/step - loss: 7.8091e-04
Epoch 8/50
3368/3368 [==============================] - 48s 14ms/step - loss: 7.5872e-04
Epoch 9/50
3368/3368 [==============================] - 48s 14ms/step - loss: 7.3889e-04
Epoch 10/50
3368/3368 [==============================] - 48s 14ms/step - loss: 7.2262e-04
Epoch 11/50
3368/3368 [==============================] - 48s 14ms/step - loss: 7.1033e-04
Epoch 12/50
3368/3368 [====

In [67]:
# demonstrate prediction
x_input = array(sensor_seq[1:15])
# x_input = x_input/maximum
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print("predicted next value: ", yhat * maximum)
print("actual next value: ", sensor_seq[15] * maximum)

predicted next value:  [[561.71326]]
actual next value:  404.0


In [68]:
# demonstrate prediction
x_input = array(sensor_seq[21:35])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print("predicted next value: ", yhat * maximum)
print("actual next value: ", sensor_seq[35] * maximum)

predicted next value:  [[97.81108]]
actual next value:  126.00000000000001
